In [1]:
#from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [1]:
%matplotlib notebook

import importlib
from datetime import datetime

#from utils36 import plots
from __future__ import division, print_function

In [11]:
import utils36
from utils36 import *
#importlib.reload(utils36)


In [8]:
from keras.utils.data_utils import get_file

In [3]:
#change slash in path to use in windows
model_path = 'data\imdb\models'

%mkdir $model_path

A subdirectory or file data\imdb\models already exists.


## Setup data

We're going to look at the IMDB dataset, which contains movie reviews from IMDB, along with their sentiment. Keras comes with some helpers for this dataset.

In [4]:
from keras.datasets import imdb
idx = imdb.get_word_index()

This is the word list:

In [5]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

...and this is the mapping from id to word

In [6]:
idx2word = {v: k for k, v in idx.items()}

We download the reviews using code copied from keras.datasets:

In [9]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')


In [12]:
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [13]:
len(x_train)

25000

Here's the 1st review. As you see, the words have been replaced by ids. The ids can be looked up in idx2word.

In [14]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

The first word of the first review is 23022. Let's see what that is.

In [15]:
idx2word[23022]

'bromwell'

Here's the whole review, mapped from ids to words.

In [16]:
' '.join([idx2word[o] for o in x_train[0]])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

The labels are 1 for positive, 0 for negative.

In [17]:
labels_train[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

Reduce vocab size by setting rare words to max index.

In [18]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

Look at distribution of lengths of sentences.

In [21]:
temp1 = map(len, trn)

In [26]:
trn[:10]

[array([4999,  309,    6,    3, 1069,  209,    9, 2175,   30,    1,  169,   55,   14,   46,   82,
        4999,   41,  393,  110,  138,   14, 4999,   58, 4477,  150,    8,    1, 4999, 4999,  482,
          69,    5,  261,   12, 4999, 4999, 2003,    6,   73, 2436,    5,  632,   71,    6, 4999,
           1, 4999,    5, 2004, 4999,    1, 4999, 1534,   34,   67,   64,  205,  140,   65, 1232,
        4999, 4999,    1, 4999,    4,    1,  223,  901,   29, 3024,   69,    4,    1, 4999,   10,
         694,    2,   65, 1534,   51,   10,  216,    1,  387,    8,   60,    3, 1472, 3724,  802,
           5, 3521,  177,    1,  393,   10, 1238, 4999,   30,  309,    3,  353,  344, 2989,  143,
         130,    5, 4999,   28,    4,  126, 4999, 1472, 2375,    5, 4999,  309,   10,  532,   12,
         108, 1470,    4,   58,  556,  101,   12, 4999,  309,    6,  227, 4187,   48,    3, 2237,
          12,    9,  215]),
 array([4999,   39, 4999,   14,  739, 4999, 3428,   44,   74,   32, 1831,   15,  150,   18

In [27]:
temp1[1]

TypeError: 'map' object is not subscriptable

In [29]:
lens = np.array(list(map(len, trn)))
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

Pad (with zero) or truncate each sentence to make consistent length.

In [30]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

This results in nice rectangular matrices that can be passed to ML algorithms. Reviews shorter than 500 words are pre-padded with zeros, those greater are truncated.

In [31]:
trn.shape

(25000, 500)

## Create simple models

### Single hidden layer NN

The simplest model that tends to give reasonable results is a single hidden layer net. So let's try that. Note that we can't expect to get any useful results by feeding word ids directly into a neural net - so instead we use an embedding to replace them with a vector of 32 (initially random) floats for each word in the vocab.

In [32]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [33]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1600100   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,760,201
Trainable params: 1,760,201
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=2, batch_size=64)

I:\ML\Anaconda3\envs\dlwin36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 6s 231us/step - loss: 0.4798 - acc: 0.7399 - val_loss: 0.2868 - val_acc: 0.8794
Epoch 2/2
25000/25000 [==============================] - 4s 152us/step - loss: 0.2047 - acc: 0.9230 - val_loss: 0.2977 - val_acc: 0.8763


The [stanford paper](http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf) that this dataset is from cites a state of the art accuracy (without unlabelled data) of 0.883. So we're short of that, but on the right track.

### Single conv layer with max pooling

A CNN is likely to work better, since it's designed to take advantage of ordered data. We'll need to use a 1D CNN, since a sequence of words is 1D.

In [39]:
conv1 = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Dropout(0.2),
    Conv1D(64, 5, padding='same', activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [40]:
conv1.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [41]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), epochs=4, batch_size=64)

I:\ML\Anaconda3\envs\dlwin36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 9s 359us/step - loss: 0.4980 - acc: 0.7150 - val_loss: 0.2791 - val_acc: 0.8860
Epoch 2/4
25000/25000 [==============================] - 8s 313us/step - loss: 0.2490 - acc: 0.9058 - val_loss: 0.2566 - val_acc: 0.8938
Epoch 3/4
25000/25000 [==============================] - 8s 314us/step - loss: 0.1994 - acc: 0.9269 - val_loss: 0.2693 - val_acc: 0.8893
Epoch 4/4
25000/25000 [==============================] - 8s 311us/step - loss: 0.1665 - acc: 0.9408 - val_loss: 0.2711 - val_acc: 0.8882


In [42]:
conv1.fit(trn, labels_train, validation_data=(test, labels_test), epochs=4, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 8s 316us/step - loss: 0.1403 - acc: 0.9514 - val_loss: 0.3104 - val_acc: 0.8852
Epoch 2/4
25000/25000 [==============================] - 8s 313us/step - loss: 0.1171 - acc: 0.9594 - val_loss: 0.3191 - val_acc: 0.8854
Epoch 3/4
25000/25000 [==============================] - 8s 314us/step - loss: 0.0949 - acc: 0.9665 - val_loss: 0.3732 - val_acc: 0.8834
Epoch 4/4
25000/25000 [==============================] - 8s 313us/step - loss: 0.0781 - acc: 0.9717 - val_loss: 0.3917 - val_acc: 0.8818


That's well past the Stanford paper's accuracy - another win for CNNs!

In [43]:
conv1.save_weights(model_path + 'conv1.h5')

In [46]:
conv1.load_weights(model_path + 'conv1.h5')

## Pre-trained vectors

You may want to look at wordvectors.ipynb before moving on.

In this section, we replicate the previous CNN, but using pre-trained embeddings.

In [51]:
glove_path = os.path.abspath('data\glove\\results')
glove_path
#%mkdir $glove_path

'I:\\ML\\courses\\deeplearning1\\nbs\\data\\glove\\results'

In [52]:
def get_glove_dataset(dataset):
    """Download the requested glove dataset from files.fast.ai
    and return a location that can be passed to load_vectors.
    """
    # see wordvectors.ipynb for info on how these files were
    # generated from the original glove data.
    md5sums = {'6B.50d': '8e1557d1228decbda7db6dfd81cd9909',
               '6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
               '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
               '6B.300d': '30290210376887dcc6d0a5a6374d8255'}
    glove_path = os.path.abspath('data\glove\\results')
    %mkdir $glove_path
    return get_file(dataset,
                    'http://files.fast.ai/models/glove/' + dataset + '.tgz',
                    cache_subdir=glove_path,
                    md5_hash=md5sums.get(dataset, None),
                    untar=True)

In [62]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb'), encoding='latin1'),
        pickle.load(open(loc+'_idx.pkl','rb'), encoding='latin1'))

In [63]:
vecs, words, wordidx = load_vectors(data_path)

The glove word ids and imdb word ids use different indexes. So we create a simple function that creates an embedding matrix using the indexes from imdb, and the embeddings from glove (where they exist).

In [64]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size, n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            # If we can't find the word in glove, randomly initialize
            emb[i] = normal(scale=0.6, size=(n_fact,))

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [65]:
emb = create_emb()

We pass our embedding matrix to the Embedding constructor, and set it to non-trainable.

In [67]:
model = Sequential([
    Embedding(vocab_size, 50, input_length=seq_len, 
              weights=[emb], trainable=False),
    Dropout(0.25),
    Conv1D(64, 5, padding='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [68]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [69]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 8s 317us/step - loss: 0.6369 - acc: 0.6275 - val_loss: 0.5410 - val_acc: 0.7385
Epoch 2/2
25000/25000 [==============================] - 7s 298us/step - loss: 0.5062 - acc: 0.7624 - val_loss: 0.4579 - val_acc: 0.8030


In [70]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 7s 297us/step - loss: 0.4574 - acc: 0.7936 - val_loss: 0.4302 - val_acc: 0.7991
Epoch 2/2
25000/25000 [==============================] - 7s 296us/step - loss: 0.4272 - acc: 0.8058 - val_loss: 0.4227 - val_acc: 0.8056


In [71]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 7s 299us/step - loss: 0.4065 - acc: 0.8187 - val_loss: 0.4044 - val_acc: 0.8196
Epoch 2/5
25000/25000 [==============================] - 7s 297us/step - loss: 0.3841 - acc: 0.8311 - val_loss: 0.3920 - val_acc: 0.8231
Epoch 3/5
25000/25000 [==============================] - 7s 298us/step - loss: 0.3688 - acc: 0.8360 - val_loss: 0.3818 - val_acc: 0.8305
Epoch 4/5
25000/25000 [==============================] - 7s 296us/step - loss: 0.3443 - acc: 0.8487 - val_loss: 0.3911 - val_acc: 0.8257
Epoch 5/5
25000/25000 [==============================] - 7s 296us/step - loss: 0.3381 - acc: 0.8531 - val_loss: 0.3774 - val_acc: 0.8344


In [72]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 7s 298us/step - loss: 0.3259 - acc: 0.8586 - val_loss: 0.3866 - val_acc: 0.8295
Epoch 2/5
25000/25000 [==============================] - 7s 297us/step - loss: 0.3117 - acc: 0.8657 - val_loss: 0.3768 - val_acc: 0.8351
Epoch 3/5
25000/25000 [==============================] - 7s 296us/step - loss: 0.3019 - acc: 0.8699 - val_loss: 0.3789 - val_acc: 0.8331
Epoch 4/5
25000/25000 [==============================] - 7s 298us/step - loss: 0.2973 - acc: 0.8742 - val_loss: 0.4292 - val_acc: 0.8126
Epoch 5/5
25000/25000 [==============================] - 7s 296us/step - loss: 0.2828 - acc: 0.8797 - val_loss: 0.4004 - val_acc: 0.8313


We already have beaten our previous model! But let's fine-tune the embedding weights - especially since the words we couldn't find in glove just have random embeddings.

In [77]:
model.layers[0].trainable=True

In [78]:
model.optimizer.lr=1e-4

In [79]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [81]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=1, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 9s 352us/step - loss: 0.1894 - acc: 0.9246 - val_loss: 0.3087 - val_acc: 0.8799


As expected, that's given us a nice little boost. :)

In [83]:
model_path

'data\\imdb\\models'

In [82]:
model.save_weights(model_path+'glove50.h5')

## Multi-size CNN

This is an implementation of a multi-size CNN as shown in Ben Bowles' [excellent blog post](https://quid.com/feed/how-quid-uses-deep-learning-with-small-data).

In [87]:
??Input

In [84]:
#from keras.layers import Merge

ImportError: cannot import name 'Merge'

We use the functional API to create multiple conv layers of different sizes, and then concatenate them.

In [88]:
graph_in = Input ((vocab_size, 50))
convs = [ ] 
for fsz in range (3, 6): 
    x = Conv1D(64, fsz, padding='same', activation="relu")(graph_in)
    x = MaxPooling1D()(x) 
    x = Flatten()(x) 
    convs.append(x)
out = keras.layers.concatenate(convs) 
graph = Model(graph_in, out) 

In [89]:
emb = create_emb()

We then replace the conv/max-pool layer in our original CNN with the concatenated conv layers.

In [91]:
model = Sequential ([
    Embedding(vocab_size, 50, input_length=seq_len, weights=[emb]),
    Dropout (0.2),
    graph,
    Dropout (0.5),
    Dense (100, activation="relu"),
    Dropout (0.7),
    Dense (1, activation='sigmoid')
    ])

In [92]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 50)           250000    
_________________________________________________________________
dropout_23 (Dropout)         (None, 500, 50)           0         
_________________________________________________________________
model_1 (Model)              multiple                  38592     
_________________________________________________________________
dropout_24 (Dropout)         (None, 48000)             0         
_________________________________________________________________
dense_17 (Dense)             (None, 100)               4800100   
_________________________________________________________________
dropout_25 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 101       
Total para

In [93]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [94]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 19s 758us/step - loss: 0.6298 - acc: 0.6200 - val_loss: 0.3489 - val_acc: 0.8598
Epoch 2/2
25000/25000 [==============================] - 18s 727us/step - loss: 0.3307 - acc: 0.8651 - val_loss: 0.2682 - val_acc: 0.8888


Interestingly, I found that in this case I got best results when I started the embedding layer as being trainable, and then set it to non-trainable after a couple of epochs. I have no idea why!

In [98]:
model.layers[0].trainable=False

In [99]:
model.optimizer.lr=1e-5

In [100]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [101]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 17s 676us/step - loss: 0.1836 - acc: 0.9322 - val_loss: 0.2769 - val_acc: 0.8970
Epoch 2/2
25000/25000 [==============================] - 16s 642us/step - loss: 0.1762 - acc: 0.9364 - val_loss: 0.2714 - val_acc: 0.8954


This more complex architecture has given us another boost in accuracy.

## LSTM

We haven't covered this bit yet!

In [108]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len, mask_zero=True, embeddings_regularizer=l2(1e-6)),
    Dropout(0.2),
    LSTM(100, implementation=2),
    Dense(1, activation='sigmoid')])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 500, 32)           160000    
_________________________________________________________________
dropout_28 (Dropout)         (None, 500, 32)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________


In [109]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=5, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 371s 15ms/step - loss: 0.4609 - acc: 0.7737 - val_loss: 0.3102 - val_acc: 0.8702
Epoch 2/5
25000/25000 [==============================] - 370s 15ms/step - loss: 0.3206 - acc: 0.8688 - val_loss: 0.3343 - val_acc: 0.8621
Epoch 3/5
25000/25000 [==============================] - 370s 15ms/step - loss: 0.2557 - acc: 0.9009 - val_loss: 0.3090 - val_acc: 0.8698
Epoch 4/5
25000/25000 [==============================] - 370s 15ms/step - loss: 0.2281 - acc: 0.9134 - val_loss: 0.3411 - val_acc: 0.8698
Epoch 5/5
25000/25000 [==============================] - 370s 15ms/step - loss: 0.1944 - acc: 0.9279 - val_loss: 0.3176 - val_acc: 0.8680
